# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Vũ Đình Chương
- MSSV: 21127236
- Lớp: 21CLC02

# Import

In [140]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [141]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng cách khác nếu cần

# Cài đặt hàm

In [142]:
# Cài đặt các hàm cần thiết ở đây
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y

        return self


    def get_params(self):
        return self.w


    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)   
    
def mae(y, y_hat):
    return np.mean(np.abs(y.ravel() - y_hat.ravel()))

def train_1(X_train_1, X_test_1, y_train):
    lr = OLSLinearRegression().fit(X_train_1, y_train)
    lr_1 = round(lr.get_params(), 3)
    y_predict_test = round(lr.predict(X_test_1), 3)
    return y_predict_test, lr_1

def printweight(weight):
    for i in range(len(weight)):
        print(f"w{i + 1}: {weight[i]}")

def kfoldsplit(X_train_1, y_train_1, k):
    size_train = len(X_train_1) // k
    X_kfold_test = []
    y_kfold_test = []

    for fold_idx in range(k):
        start_idx = fold_idx * size_train
        end_idx = (fold_idx + 1) * size_train
        if fold_idx < k - 1:
            X_kfold_test.append(X_train_1.iloc[start_idx:end_idx])
            y_kfold_test.append(y_train_1.iloc[start_idx:end_idx])
        if fold_idx >= k - 1: 
            X_kfold_test.append(X_train_1.iloc[start_idx:])
            y_kfold_test.append(y_train_1.iloc[start_idx:])

    return X_kfold_test, y_kfold_test

def kfold(X_train_1, y_train_1, k):
    X_kfold_test, y_kfold_test = kfoldsplit(X_train_1, y_train_1, k)
    mae_arr = np.zeros(X_train_1.shape[1])
    
    for i in range(k):
        for col_idx in range(X_train_1.shape[1]):
            idx = X_kfold_test[i].index
            x_temp_train = X_train_1.iloc[:, col_idx:col_idx + 1]
            x_temp_test = X_kfold_test[i].iloc[:, col_idx:col_idx + 1]
            y_temp_predict, _ = train_1(x_temp_train.drop(idx), x_temp_test, y_train_1.drop(idx))
            mae_arr[col_idx] += mae(y_kfold_test[i], y_temp_predict)
    mae_arr /= k
    return mae_arr, X_train_1.columns[np.argmin(mae_arr)]

def detect_outliers(data):
    outlier_percents = {}
    for column in data.columns:
        if data[column].dtype != object:
            q1 = np.quantile(data[column], 0.25)
            q3 = np.quantile(data[column], 0.75)
            iqr = q3 - q1
            upper_bound = q3 + (1.5 * iqr)
            lower_bound = q1 - (1.5 * iqr)
            outliers = data[(data[column] > upper_bound) | (data[column] < lower_bound)][column]
            outlier_percentage = len(outliers) / len(data[column]) * 100
            outlier_percents[column] = outlier_percentage
            outlier_dataframe = pd.DataFrame(data = outlier_percents.values() ,index=outlier_percents.keys() ,columns=['Outlier_percentage'])
    
    return outlier_dataframe.sort_values(by = 'Outlier_percentage', ascending = False)



# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [143]:
# Phần code cho yêu cầu 1a
X_train_1a = X_train.iloc[:, :11]
X_test_1a = X_test.iloc[:, :11]
y_predict_test_1a, lr_1a = train_1(X_train_1a, X_test_1a, y_train)
printweight(lr_1a)



w1: -22756.513
w2: 804.503
w3: 1294.655
w4: -91781.898
w5: 23182.389
w6: 1437.549
w7: -8570.662
w8: 147.858
w9: 152.888
w10: 117.222
w11: 34552.286


In [144]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
mae_1a =  mae(y_test, y_predict_test_1a)
print('MAE:', mae_1a)

MAE: 104863.77754133331


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = -22756.513*Gender + 804.503*10percentage +  1294.655*12percentage \\+ (-91781.898)*CollegeTier + (-23182.389)*Degree +  1437.549*collegeGPA \\+ (-8570.662)*CollegeCityTier + 147.858*English + 152.888*Logical \\+ 117.222*Quant + 34552.286*Domain$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [145]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
feature_1b = ['conscientiousness', 'agreeableness', 'extraversion', 'nueroticism', 'openess_to_experience','Salary']
train_1b = train.loc[:, feature_1b].sample(frac=1).reset_index(drop=True)
X_train_1b = train_1b.iloc[:, :-1]
y_train_1b = train_1b.iloc[:, -1]
mae_arr_1b, best_personality_feature_model = kfold(X_train_1b, y_train_1b, 5)
data = {'Feature Name': feature_1b[:-1], 'MAE': mae_arr_1b}
df = pd.DataFrame(data)
df.index = pd.RangeIndex(stop=df.shape[0]) + 1
display(df)
print('Best personality feature model:', best_personality_feature_model)

,Feature Name,MAE
1,conscientiousness,306113.167478
2,agreeableness,300908.734761
3,extraversion,307258.219043
4,nueroticism,299369.912854
5,openess_to_experience,303056.981407


Best personality feature model: nueroticism


In [146]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
f_train_1b = X_train.loc[:, [best_personality_feature_model]]
f_test_1b = X_test.loc[:, [best_personality_feature_model]]
y_predict_test_1b, lr_1b = train_1(f_train_1b, f_test_1b, y_train)
printweight(lr_1b)


w1: -56546.304


In [147]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
mae_1b =  mae(y_test, y_predict_test_1b)
print('MAE:', mae_1b)

MAE: 291019.69318933337


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = (-56546.304)*nueroticism$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [148]:
# Phần code cho yêu cầu 1c
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
feature_1c = ['English','Logical','Quant','Salary']
train_1c = train.loc[:, feature_1c].sample(frac=1).reset_index(drop=True)
X_train_1c = train_1c.iloc[:, :-1]
y_train_1c = train_1c.iloc[:, -1]
mae_arr_1c, best_skill_feature_model = kfold(X_train_1c, y_train_1c, 5)
data = {'Feature Name': feature_1c[:-1], 'MAE': mae_arr_1c}
df = pd.DataFrame(data)
df.index = pd.RangeIndex(stop=df.shape[0]) + 1
display(df)
print('Best skill feature model:', best_skill_feature_model)

,Feature Name,MAE
1,English,121913.655721
2,Logical,120345.220285
3,Quant,118265.099776


Best skill feature model: Quant


In [149]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
f_train_1c = X_train.loc[:, [best_skill_feature_model]]
f_test_1c = X_test.loc[:, [best_skill_feature_model]]
y_predict_test_1c, lr_1c = train_1(f_train_1c, f_test_1c, y_train)
printweight(lr_1c)

w1: 585.895


In [150]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model
mae_1c =  mae(y_test, y_predict_test_1c)
print('MAE:', mae_1c)

MAE: 106819.57762533335


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = 585.895*Quant$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

In [204]:
MAE_arr, best_feature = kfold(X_train, y_train, 5)
SortedFeatures = df.sort_values(["MAE"], ignore_index=True)
data = {'Feature Name': X_train.columns, 'MAE': MAE_arr}
df = pd.DataFrame(data)
df.index = pd.RangeIndex(stop=df.shape[0]) + 1
SortedFeatures = df.sort_values(["MAE"], ignore_index=True)
display(df)
print('Best feature:', best_feature)

,Feature Name,MAE
1,Gender,150490.539245
2,10percentage,119007.839359
3,12percentage,120142.226628
4,CollegeTier,133602.488494
5,Degree,137556.983498
6,collegeGPA,121590.847634
7,CollegeCityTier,251756.493443
8,English,121987.765135
9,Logical,120375.764674
10,Quant,118224.571054


Best feature: Quant


In [243]:
# Trình bày các phần tìm ra mô hình

# Chọn các đặc trưng dựa trên mae: chọn các đặc trưng cho mae dưới 160000
best_feature_1d = ['Quant', '10percentage', '12percentage', 'Logical', 'collegeGPA', 'English', 'CollegeTier', 'Degree', 'ComputerProgramming', 'Salary']
# Mô hình gốc
X0_train_1d = train.loc[:, best_feature_1d]

# Mô hình có đặc trưng tốt nhất bình phương
X1_train_1d = train.loc[:, best_feature_1d]
X1_train_1d['Quant'] = X1_train_1d['Quant'] ** 2

# Mô hình có đặc trưng tốt nhất mũ 1/2
X2_train_1d = train.loc[:, best_feature_1d]
X2_train_1d['Quant'] = X2_train_1d['Quant'] ** 0.5

# Mô hình có đặc trưng tốt nhất mũ 1/4
X3_train_1d = train.loc[:, best_feature_1d]
X3_train_1d['Quant'] = X3_train_1d['Quant'] ** 0.25

ModelNames = pd.Series([
    "Mô hình gốc",
    "Mô hình có đặc trưng tốt nhất bình phương",
    "Mô hình có đặc trưng tốt nhất mũ 1/2",
    "Mô hình có đặc trưng tốt nhất mũ 1/4",
])

ModelData = [
    X0_train_1d,
    X1_train_1d,
    X2_train_1d,
    X3_train_1d
]


## Thử nghiệm, so sánh các mô hình

In [244]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu
mae_arr_1d = []
train_1d = train.sample(frac=1).reset_index(drop=True)
for data in ModelData:
    X_feature_1d = data.iloc[:, :-1]
    y_feature_1d = data.iloc[:, -1]
    mae_arr, _ = kfold(X_feature_1d, y_feature_1d, 5)
    mae_arr_1d.append(np.mean(mae_arr))

my_best_model = ModelNames[np.argmin(mae_arr_1d)]
data = {'Model Name': ModelNames, 'MAE': mae_arr_1d}
df = pd.DataFrame(data)
df.index = pd.RangeIndex(stop=df.shape[0]) + 1
display(df)
print('My best model:', my_best_model)

,Model Name,MAE
1,Mô hình gốc,127631.142759
2,Mô hình có đặc trưng tốt nhất bình phương,129231.720815
3,Mô hình có đặc trưng tốt nhất mũ 1/2,127553.420557
4,Mô hình có đặc trưng tốt nhất mũ 1/4,127765.505828


My best model: Mô hình có đặc trưng tốt nhất mũ 1/2


In [253]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
X_test_1d = test.loc[:, best_feature_1d]
X_test_1d['Quant'] = X_test_1d['Quant'] ** 0.5
y_predict_test_1d, lr_1d = train_1(X2_train_1d.iloc[:, :-1], X_test_1d.iloc[:, :-1], y_train)
printweight(lr_1d)

w1: 6211.129
w2: 565.568
w3: 1066.678
w4: 138.269
w5: 945.317
w6: 131.441
w7: -102989.132
w8: 9029.608
w9: 85.731


In [250]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
mae_1d =  mae(y_test, y_predict_test_1d)
print('MAE:', round(mae_1d, 3))

MAE: 104776.914


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = 6211.129*Quant + 565.568*10percentage + 1066.678*12percentage + 138.269*Logical +945.317*collegeGPA + 131.441*English \\ + (-102989.132)*CollegeTier + 9029.608*Degree + 85.731*ComputerProgramming$$